# Azure AI Agent with Existing Agent Example

This notebook demonstrates working with pre-existing Azure AI Agents by providing agent IDs, showing agent reuse patterns for production scenarios.

## Features Covered:
- Creating and managing persistent Azure AI agents
- Working with existing agent IDs
- Agent lifecycle management (create, use, delete)
- Production patterns for agent reuse
- Manual cleanup of agents

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with an existing Azure AI agent by connecting to it using its agent ID.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [ ]:
#!about
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.*"
#r "nuget: Microsoft.Agents.AI.AzureAI, 1.0.0-preview.*"
#r "nuget: dotenv.net"

#pragma warning disable CA1812
#pragma warning disable MEAI001

using Azure.Identity;
using Microsoft.Agents.AI;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Microsoft.Extensions.AI;
using dotenv.net;
using System.IO;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Text.Json;
using OpenAI.Assistants;
using OpenAI.Responses;

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [ ]:
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".", "..", "..","..", ".env") })); 
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeploymentName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
// Create the Credentials
var credentialOptions = new InteractiveBrowserCredentialOptions
{
    TenantId = tenantId
};
var credential = new InteractiveBrowserCredential(credentialOptions);
const string AgentName = "DemoFoundryAgent";

AIProjectClient aiProjectClient = new AIProjectClient( new Uri(projectEndpoint),   credential);

## Create your agent

1. Login to the [Foundry portal](https://ai.azure.com)
2. Select the "New Foundry" toggle at the top of the screen to go to the new foundry portal if you're not already there
3. Select the project you wish to use in the Foundry (or create a new project if you don't have one.)
4. Click on `Build` at the top-nav.
![New Foundry Build Menu](../../../images/new_foundry_build_menu.png)
5. Click on the `Create agent` button
6. Name your agent `DemoFoundryAgent` (or change the name in the cell above to what you wish to name your agent.)
7. In the playground, create your instructions (you can use a simple instruction of "You are a helpful Weather Agent.")
8. Select the model you wish the agent to use.
9. Click on `Tools` to expand the tools configuration panel.
10. Click on `Add`, and then `Add a new tool`
11. Select `Custom` from the tool tabs, and then `OpenAPI tool` and click `Create`
12. Name the tool, enter a helpful description such as `This tool retrieves the current weather for a given location`.
13. Go to the sample spec [here](https://github.com/microsoft-foundry/foundry-samples/blob/42b1200b5fa1ca6d10ed93f58a478a3fcd1d6dad/samples-classic/python/getting-started-agents/openapi/weather_openapi.json) and copy the raw contents into the `OpenAPI 3.0+ Schema` box, the click on `Create tool`
![Example Foundry Tool](../../../images/foundry_tool_config.png)

## Define Function Tools and AI Agent

Define our tools and agent for use

In [ ]:
[Description("Get the current weather for a given location.")]
string GetWeather([Description("The location to get the weather for.")]string location)
{
    // Simulate getting weather data
    var conditions = new[] { "sunny", "cloudy", "rainy", "windy", "stormy" };
    var random = new Random();
    var condition = conditions[random.Next(conditions.Length)];
    
    return $"🌤️ The weather in {location} is {condition} with a high of {random.Next(60, 101)}°F.";
}

var tool = AIFunctionFactory.Create(GetWeather);

var agent = await aiProjectClient.CreateAIAgentAsync(
    name: AgentName,
    instructions: "You are a helpful weather agent.",
    model: modelDeploymentName!,
    tools: [tool]
);

## Retrieve and use a Foundry Agent

Here's an example of how to work with an agent that was created through the Foundry portal:

In [ ]:
async Task<ChatClientAgent> GetExistingAgent()
{
    // Retrieve the existing agent by name
    var tool = AIFunctionFactory.Create(GetWeather);
    var agent = await aiProjectClient.GetAIAgentAsync(name: AgentName);
   
    return agent;
}


var agent = await GetExistingAgent();
Console.WriteLine($"👤 User: What's the weather like in London?");
var thread = agent.GetNewThread();
var response = await agent.RunAsync("What's the weather like in London?", thread);
Console.WriteLine($"🤖 Agent: {response.Text}");